In [39]:
import glob
import cv2
import matplotlib.pyplot as plt
import numpy as np

criteria = (cv2.TERM_CRITERIA_EPS + cv2.TERM_CRITERIA_MAX_ITER, 30, 0.001)

images = glob.glob('../calibration-images/*.jpg')


In [40]:
patternSize = (7,7)
squareSize = (10)
imgSize = (3280,2464)

def construct3DPoints(patternSize,squareSize):
    X = np.zeros((patternSize[0]*patternSize[1],3), np.float32)
    X[:,:2] = np.mgrid[0:patternSize[0],0:patternSize[1]].T.reshape(-1,2)
    X = X * squareSize
    return X

boardPoints = construct3DPoints(patternSize,squareSize)
worldPoints = []
imagePoints = [] 

In [41]:
counter = 0
for fname in images:
    img = cv2.imread(fname)
    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    ret, corners = cv2.findChessboardCornersSB(gray, patternSize, None)
    if ret == True:
        print("Corners found in image" + str(fname)) #- see if corners are found 
        imagePoints.append(corners)
        worldPoints.append(boardPoints)
        counter+=1

print("using " + str(counter) + " images")

Corners found in image../calibration-images/image5.jpg
Corners found in image../calibration-images/image3.jpg
Corners found in image../calibration-images/image2.jpg
Corners found in image../calibration-images/image4.jpg
Corners found in image../calibration-images/image1.jpg
Corners found in image../calibration-images/image7.jpg
Corners found in image../calibration-images/image6.jpg
using 7 images


In [42]:
flagsCalib = cv2.CALIB_RATIONAL_MODEL
ret, cameraMatrix, k, rvecs, tvecs = cv2.calibrateCamera(worldPoints, imagePoints, imgSize, None, None,flags=flagsCalib)

print("Using "+str(counter)+" of "+str(len(images))+" images")
print("RMS re-projection error:", ret)
print("Camera Matrix:\n", cameraMatrix)
print("Distortion Parameters:\n", k)

Using 7 of 7 images
RMS re-projection error: 0.2823487864500124
Camera Matrix:
 [[1.12310203e+03 0.00000000e+00 6.33336776e+02]
 [0.00000000e+00 1.12297398e+03 3.61203581e+02]
 [0.00000000e+00 0.00000000e+00 1.00000000e+00]]
Distortion Parameters:
 [[-8.84147696e+00  4.99414589e+01  4.50787979e-04 -1.85475747e-03
  -8.76951801e+01 -8.81474874e+00  4.96391196e+01 -8.68913919e+01
   0.00000000e+00  0.00000000e+00  0.00000000e+00  0.00000000e+00
   0.00000000e+00  0.00000000e+00]]


In [ ]:
def plot_image_points(image_points):
    """
    Plota os pontos 2D na imagem.

    :param image_points: Array de pontos 2D projetados na imagem.
    """

    if image_points.shape[1] != 2:
        raise ValueError("A matriz de image_points deve ter duas colunas (X e Y).")
    plt.figure(figsize=(8, 6))
    plt.scatter(image_points[:, 0], image_points[:, 1], c='b', marker='o', label='Pontos dectados')


    plt.xlabel('Coordenada X na Imagem')
    plt.ylabel('Coordenada Y na Imagem')
    plt.title('Pontos 2D na Imagem')
    plt.grid()
    plt.legend()
    plt.show()

def plot_points(image_points, projected_points, image_path=None):
    """
    Plota os pontos detectados na imagem e os pontos projetados pela câmera.

    :param image_points: Pontos 2D observados nas imagens reais.
    :param projected_points: Tupla de arrays com pontos projetados a partir da projeção dos pontos 3D.
    :param image_path: Caminho para a imagem de fundo (opcional).
    """
    plt.figure()

    # Plota os pontos detectados na imagem (pontos originais) em azul
    plt.scatter(image_points[:, 0], image_points[:, 1], color='blue', label='Pontos Originais')

    for points in projected_points:
        # Garante que points seja bidimensional (Nx2)
        if points.shape[1] == 1:
            points = points.reshape(-1, 2)
        plt.scatter(points[:, 0], points[:, 1], color='red', label='Pontos Projetados')

    # Define rótulos e legenda
    plt.xlabel('Coordenada X')
    plt.ylabel('Coordenada Y')
    plt.legend()

    # Se um caminho de imagem for fornecido, plote a imagem de fundo
    if image_path:
        img = plt.imread(image_path)
        plt.imshow(img, extent=[0, img.shape[1], 0, img.shape[0]], alpha=0.5)

    plt.show()


imagePoints = []
image_test = cv2.imread('../calibration-images/image1.jpg')
gray = cv2.cvtColor(image_test, cv2.COLOR_BGR2GRAY)
ret, corners = cv2.findChessboardCornersSB(gray, patternSize, None)
if ret == True:
    print("Corners found in image") #- see if corners are found 
    imagePoints.append(corners)

worldPoints = np.array(worldPoints)

projectedImagePoints = cv2.projectPoints(objectPoints = boardPoints.reshape(-1,1,3),
                                         rvec = rvecs[0].reshape(-1,3),
                                         tvec = tvecs[0].reshape(-1,3),
                                         cameraMatrix = cameraMatrix,
                                        distCoeffs = k)

#projectedImagePoints  = projectedImagePoints.reshape(-1,2).astype(float)

imagePoints = np.array(imagePoints)
imagePoints = imagePoints.reshape(-1,2)
print(projectedImagePoints)

plot_points(imagePoints,projectedImagePoints)